In [1]:
import numpy as np
import update_register as ur
from aiapy.calibrate import register, update_pointing
import sunpy.map

from astropy.io import fits
import astropy.units as u
from sunpy.map.sources.sdo import AIAMap, HMIMap
from sunpy.map import map_edges
try:
    import cv2
    HAS_CV2 = True
except ImportError:
    HAS_CV2 = False

try:
    import cupy
    from cupyx.scipy.ndimage import affine_transform as cupy_affine_transform
    HAS_CUPY = True
except ImportError:
    HAS_CUPY = False

In [4]:
%%time
path = './AIA_data/aia_lev1_171a_2017_09_10t01_17_09_35z_image_lev1.fits'
m = sunpy.map.Map(path)
m_up = update_pointing(m)
smap = m_up
missing = None
interp = cv2.INTER_CUBIC

CPU times: user 345 ms, sys: 11.7 ms, total: 356 ms
Wall time: 2.26 s


In [12]:
%%time
if not isinstance(smap, (AIAMap, HMIMap)):
    raise ValueError("Input must be an AIAMap or HMIMap.")

# Target scale is 0.6 arcsec/pixel, but this needs to be adjusted if the
# map has already been rescaled.
if ((smap.scale[0] / 0.6).round() != 1.0 * u.arcsec / u.pix
        and smap.data.shape != (4096, 4096)):
    scale = (smap.scale[0] / 0.6).round() * 0.6 * u.arcsec
else:
    scale = 0.6 * u.arcsec  # pragma: no cover # can't test this because it needs a full res image
scale_factor = smap.scale[0] / scale

missing = smap.min() if missing is None else missing

scale_factor=scale_factor.value
angle = None

CPU times: user 590 µs, sys: 0 ns, total: 590 µs
Wall time: 593 µs


In [26]:
%%time
if missing is None:
    missing = smap.min()

if angle is None:
    ang = -smap.meta['CROTA2']
elif angle is not None:
    ang = angle

# convert angle to radian
c = np.cos(np.deg2rad(ang))
s = np.sin(np.deg2rad(ang))
rmatrix = np.array([[c, -s],
                    [s, c]])

array_center = (np.array(smap.data.shape)[::-1] - 1) / 2.0

# Copy meta data
new_meta = smap.meta.copy()

extent = np.max(np.abs(np.vstack((smap.data.shape @ rmatrix,
                                  smap.data.shape @ rmatrix.T))), axis=0)

# Calculate the needed padding or unpadding
diff = np.asarray(np.ceil((extent - smap.data.shape) / 2), dtype=int).ravel()
# Pad the image array
pad_x = int(np.max((diff[1], 0)))
pad_y = int(np.max((diff[0], 0)))

new_meta['crpix1'] += pad_x
new_meta['crpix2'] += pad_y

new_data = np.pad(smap.data,
                  ((pad_y, pad_y), (pad_x, pad_x)),
                  mode='constant',
                  constant_values=(missing, missing))

pixel_array_center = (np.flipud(new_data.shape) - 1) / 2.0

# Create a temporary map so we can use it for the data to pixel calculation.
temp_map = smap._new_instance(new_data, new_meta, smap.plot_settings)

#this is same as `reference_pixel` in R. Attie original scale_rotate
pixel_rotation_center = u.Quantity(temp_map.world_to_pixel(smap.reference_coordinate, origin=0)).value
pixel_center = pixel_rotation_center

del temp_map

# DO CV THING HERE
padded_array_center = (np.array(new_data.shape)[::-1] - 1) / 2.0
padded_reference_pixel = pixel_rotation_center + np.array([pad_x, pad_y])
rmatrix_cv = cv2.getRotationMatrix2D((padded_reference_pixel[0], padded_reference_pixel[1]), ang, scale_factor)

# Adding extra shift to recenter:
# move image so the reference pixel aligns with the center of the padded array
shift = padded_array_center - padded_reference_pixel
rmatrix_cv[0, 2] += shift[0]
rmatrix_cv[1, 2] += shift[1]

#cast new_data to float64, then warpAffine it
new_data = new_data.astype(np.float64, casting='safe')

CPU times: user 111 ms, sys: 11.6 ms, total: 123 ms
Wall time: 122 ms


In [27]:
rotated_image = cv2.warpAffine(new_data, rmatrix_cv, new_data.shape, cv2.INTER_LINEAR)
rotated_image = cv2.warpAffine(new_data, rmatrix_cv, new_data.shape, cv2.INTER_CUBIC)

In [29]:
%timeit rotated_image = cv2.warpAffine(new_data, rmatrix_cv, new_data.shape, cv2.INTER_LINEAR)

55.4 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%timeit rotated_image = cv2.warpAffine(new_data, rmatrix_cv, new_data.shape, cv2.INTER_CUBIC)

54.9 ms ± 652 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%time

new_reference_pixel = pixel_array_center

# The FITS-WCS transform is by definition defined around the
# reference coordinate in the header.
lon, lat = smap._get_lon_lat(smap.reference_coordinate.frame)
rotation_center = u.Quantity([lon, lat])

# Define the new reference_pixel
new_meta['crval1'] = rotation_center[0].value
new_meta['crval2'] = rotation_center[1].value
new_meta['crpix1'] = new_reference_pixel[0] + 1  # FITS pixel origin is 1
new_meta['crpix2'] = new_reference_pixel[1] + 1  # FITS pixel origin is 1

# Unpad the array if necessary
unpad_x = -np.min((diff[1], 0))
if unpad_x > 0:
    new_data = new_data[:, unpad_x:-unpad_x]
    new_meta['crpix1'] -= unpad_x
unpad_y = -np.min((diff[0], 0))
if unpad_y > 0:
    new_data = new_data[unpad_y:-unpad_y, :]
    new_meta['crpix2'] -= unpad_y

# Calculate the new rotation matrix to store in the header by
# "subtracting" the rotation matrix used in the rotate from the old one
# That being calculate the dot product of the old header data with the
# inverse of the rotation matrix.
pc_C = np.dot(smap.rotation_matrix, np.linalg.inv(rmatrix))
new_meta['PC1_1'] = pc_C[0, 0]
new_meta['PC1_2'] = pc_C[0, 1]
new_meta['PC2_1'] = pc_C[1, 0]
new_meta['PC2_2'] = pc_C[1, 1]

# Update pixel size if image has been scaled.
if scale_factor != 1.0:
    new_meta['cdelt1'] = (smap.scale[0] / scale_factor).value
    new_meta['cdelt2'] = (smap.scale[1] / scale_factor).value

# Remove old CROTA kwargs because we have saved a new PCi_j matrix.
new_meta.pop('CROTA1', None)
new_meta.pop('CROTA2', None)
# Remove CDi_j header
new_meta.pop('CD1_1', None)
new_meta.pop('CD1_2', None)
new_meta.pop('CD2_1', None)
new_meta.pop('CD2_2', None)

# Create new map with the modification
new_map = smap._new_instance(new_data, new_meta, smap.plot_settings)


CPU times: user 43.4 ms, sys: 0 ns, total: 43.4 ms
Wall time: 33.8 ms


## - Total Sunpy overhead: ~140ms; with Sunpy data load: >2s
## - openCV interpolation: ~55 ms